In [1]:
import pandas as pd 
from lib.graph import Graph

from utils import df_tools
from utils import cypher
from utils import neo_utils

In [2]:
pets = pd.read_csv('pets.csv')
pets

,name,species,color,age,behavior
0,Ralph,Dog,black,10.0,NaN
1,Pip,Cat,yellow,6.0,good
2,Babe,Pig,NaN,3.0,NaN
3,Bubbles,Fish,red,NaN,acceptable
4,Freckles,Horse,brown,NaN,NaN


## Set up Graph

In [3]:
uri = 'bolt://localhost:7687'
auth = ('neo4j', 'neonpandas')

In [4]:
graph = Graph(uri, auth)

### Creates Nodes via DataFrame

In [ ]:
graph.create_nodes(pets, attr='Pet', use_column='species')

### Create Constraints via DataFrame

In [ ]:
constraints = pd.DataFrame([
    {'attr': 'Pet', 'property': 'name'}, 
    {'attr': 'Owner', 'property': 'name'}
])
constraints

In [ ]:
# create from dataframe
graph.create_node_constraints(constraints)

### Create Edges

In [ ]:
edges = pd.DataFrame([
    {'src': 'Ralph', 'rel_type': 'SHARES_OWNER', 'dest': 'Bubbles'},
    {'src': 'Pip', 'rel_type': 'SHARES_OWNER', 'dest': 'Babe'},
    {'src': 'Ralph', 'rel_type': 'SHARES_OWNER', 'dest': 'Freckles'}
])
edges

In [ ]:
query = """UNWIND $edges AS edge
        MATCH (s:Pet {name: edge.src})
        MATCH (d:Pet {name: edge.dest})
        WITH s,d,edge
        CALL apoc.merge.relationship(s, edge.rel_type, {}, {}, d) YIELD rel
        RETURN COUNT(rel)"""

In [ ]:
graph.create_relationships(edges, query)

### Node Matching

In [5]:
new_pets = pd.DataFrame([
    {'name': 'Betsy', 'age': 2, 'species': 'Cow'},
    {'name': 'Carrie', 'species': 'Rabbit'}
])
all_pets = pd.concat([pets, new_pets], sort=False)
all_pets

,name,species,color,age,behavior
0,Ralph,Dog,black,10.0,NaN
1,Pip,Cat,yellow,6.0,good
2,Babe,Pig,NaN,3.0,NaN
3,Bubbles,Fish,red,NaN,acceptable
4,Freckles,Horse,brown,NaN,NaN
0,Betsy,Cow,NaN,2.0,NaN
1,Carrie,Rabbit,NaN,NaN,NaN


In [23]:
def node_match(nodes:pd.DataFrame, on:str, attr:str, return_all=True) -> pd.DataFrame:
    query = cypher.bulk_node_match_query(attr=attr, field=on)
    print(query)
    nodes = df_tools.convert_to_records(nodes)
    with graph.driver.session() as session:
        result = session.read_transaction(neo_utils._bulk_node_match, nodes, query)
    return result

In [24]:
test = node_match(all_pets, on='name', attr='Pet')

UNWIND $nodes AS node
    MATCH (n:Pet { name: node.name })
    RETURN n.name AS name


In [41]:
def print_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
                         "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])
        
def get_nodes(tx):
    return tx.run('MATCH (n) RETURN n')

In [69]:
## this works!
session = graph.driver.session()
result = session.run('MATCH (n) RETURN n')

In [71]:
[n for n in result.graph().nodes]

[<Node id=10 labels=frozenset({'Dog', 'Pet'}) properties={'color': 'black', 'name': 'Ralph', 'age': 10.0}>,
 <Node id=11 labels=frozenset({'Cat', 'Pet'}) properties={'name': 'Pip', 'color': 'yellow', 'behavior': 'good', 'age': 6.0}>,
 <Node id=12 labels=frozenset({'Pig', 'Pet'}) properties={'name': 'Babe', 'age': 3.0}>,
 <Node id=13 labels=frozenset({'Fish', 'Pet'}) properties={'color': 'red', 'name': 'Bubbles', 'behavior': 'acceptable'}>,
 <Node id=14 labels=frozenset({'Horse', 'Pet'}) properties={'color': 'brown', 'name': 'Freckles'}>]

In [65]:
## need to construct a dataframe from these results
## change attributes to labels (re: terminology)
## re-work session/runs/transactions using above approach
## push repo to git (after creating README and .gitignore)

In [66]:
test[0]

IndexError: list index out of range

In [ ]:
test[0].